In [ ]:
%tensorflow_version 2.x


In [ ]:
import pandas as pd 
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
# import requests
from io import StringIO
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
# from chart_studio.plotly import plot_mpl
# import cufflinks as cf
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from scipy import stats
# import seaborn as sns


Authorization Code = 4/wwGLqpIPtyEx8LBrxVOPkkOBWzz7AjSVdkXV6xEyTjmrumQ9w1Rpd3c

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [ ]:
cd drive/

In [ ]:
pwd

In [ ]:
cd My  Drive


In [ ]:
cd Major_Project/src/

In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
df = pd.read_csv('/home/deaddemocracy/Documents/Major_Project/Extract_Feb18_28_1PM/Sensor1/sensor1_pp.csv', index_col = 0)

In [ ]:
df.head()


In [ ]:
df.index = pd.to_datetime(df.index)

In [ ]:
ax_data = df['az(g)'].resample('1min').mean()

In [ ]:
type(ax_data)

In [ ]:
train_size = int(len(ax_data) * 0.8)

In [ ]:
test_size = len(ax_data) - train_size

In [ ]:
train, test = ax_data[0:train_size], ax_data[train_size:len(df)]

In [ ]:
train.tail()

In [ ]:
print(len(train), len(test))

In [ ]:
def create_dataset(dataset,look_back=1):
    Xs, ys = [], []
    dataset = np.array(dataset)
    for i in range(len(dataset) - look_back - 1):
        v= dataset[i:(i + look_back)]
        Xs.append(v)
        ys.append(dataset[(i + look_back)])
    return np.array(Xs), np.array(ys)

In [ ]:
look_back = 10

In [ ]:
train.shape

In [ ]:
X_train, y_train = create_dataset(train, look_back)
X_test, y_test = create_dataset(test, look_back)

print(X_train.shape, y_train.shape)

In [ ]:
# # X_train[1:5]
# X_test[1:5]

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0],1,X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0],1,X_test.shape[1]))

In [ ]:
# X_train[-5:]

**Stationarity Test**

In [ ]:
stat, p = stats.normaltest(ax_data)
print('Statistics=%.3f, p=%.3f' % (stat, p))
alpha = 0.05
if p > alpha:
    print('Data looks Gaussian (fail to reject H0)')
else:
    print('Data does not look Gaussian (reject H0)')

In [ ]:
# sns.distplot(ax_data);
# print( 'Kurtosis of normal distribution: {}'.format(stats.kurtosis(ax_data)))
# print( 'Skewness of normal distribution: {}'.format(stats.skew(ax_data)))

# **Modelling Using LSTM**

In [ ]:
model = keras.Sequential()
model.add(keras.layers.LSTM(units = 128, input_shape = (1,look_back)))


In [ ]:
model.add(keras.layers.Dense(units=1))
model.compile(loss='mean_squared_error',optimizer=keras.optimizers.Adam(0.001))
model.fit(X_train, y_train, epochs = 10, batch_size=1, verbose=2)

In [ ]:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)


In [ ]:
test_predict[:10]

In [ ]:
print('Train Mean Absolute Error:', keras.losses.MAE(y_train[0], train_predict[:,0]))
print('Train Root Mean Squared Error:',np.sqrt(keras.losses.MSE(y_train[0], train_predict[:,0])))
print('Test Mean Absolute Error:', keras.losses.MAE(y_test[0], test_predict[:,0]))
print('Test Root Mean Squared Error:',np.sqrt(keras.losses.MSE(y_test[0], test_predict[:,0])))

In [ ]:
aa=[x for x in range(len(train_predict))]
# plt.figure(figsize=(8,4))
# plt.plot(aa, y_train, marker='.', label="actual")
# plt.plot(aa, train_predict, 'r', label="prediction")
# # plt.tick_params(left=False, labelleft=True) #remove ticks
# #plt.tight_layout()
# plt.subplots_adjust(left=0.07)
# plt.ylabel('az(g)', size=15)
# plt.xlabel('Time step', size=15)
# plt.legend(fontsize=15)
# plt.show();

actual_plot = mean = go.Scatter(x=aa,y=y_train,name="actual", line=dict(color='chartreuse'))
train_predict_plot = go.Scatter(x=aa,y=train_predict,name="prediction",line=dict(color='black'))
layout = go.Layout(yaxis=dict(range=[1.0135,1.019]))
fig4 = go.Figure(data = actual_plot,layout=layout)
fig4.add_trace(train_predict_plot)
fig4.show()

In [ ]:
layout = go.Layout(yaxis=dict(range=[1.0135,1.019]))
fig2 = go.Figure(data = original,layout=layout)
fig2.add_trace(mean)
fig2.add_trace(std)
fig2.show()

# **ARIMA**

In [ ]:
from statsmodels.tsa.stattools import adfuller
from numpy import log 
from matplotlib.pylab import rcParams
rcParams['figure.figsize']=(15,6)

**Stationarity Test**

In [ ]:
def stationarity_check(timeseries):
  # Rolling statistics
  rolling_mean = timeseries.rolling(window = 20, center=False).mean()
  rolling_std = timeseries.rolling(window = 20, center=False).std()

  #Plot rolling statistics
  original = go.Scatter(x=timeseries.index,y=timeseries.values)
  mean = go.Scatter(x=rolling_mean.index,y=rolling_mean.values, line=dict(color='chartreuse'))
  std = go.Scatter(x=rolling_std.index,y=rolling_std.values, line=dict(color='black'))

  print('Dickey Fuller Test:')
  di_fu_test = adfuller(timeseries,autolag='AIC')
  df_results = pd.Series(di_fu_test[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
  for key,value in di_fu_test[4].items():
        df_results['Critical Value (%s)'%key] = value
  print (df_results)
 
  return original,mean,std

In [ ]:
original,mean,std = stationarity_check(ax_data)
mean

*Since the p-value is 0.01 i.e.99% confidence, the time series is stationary*

In [ ]:
# ax_data.max()

In [ ]:
layout = go.Layout(yaxis=dict(range=[1.0135,1.019]))
fig2 = go.Figure(data = original,layout=layout)
fig2.add_trace(mean)
fig2.add_trace(std)
fig2.show()

In [ ]:
# fig = go.Figure()
# fig.add_trace(go.Scatter(x = ax_data.index, y = ax_data.values))
# fig.show()


**Apply non linear transformation for stationarity test**

In [ ]:
az_data_log = pd.Series(np.log(az_data.values),az_data.index)

In [ ]:
original_log,mean_log,std_log = stationarity_check(az_data_log)

In [ ]:
fig2 = go.Figure(data = original_log)
fig2.add_trace(mean_log)
fig2.add_trace(std_log)
fig2.show()

**Differencing the series and Autocorrelation plot - Unnecessary**


In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
plt.rcParams.update({'figure.figsize':(9,7), 'figure.dpi':120})

In [ ]:
fig,axes = plt.subplots(3,2,sharex=True);

diff_first = np.diff(ax_data3.values)
diff_second = np.diff(diff_first)

ax_data.values
# diff_first 
# diff_second

#Original Series
axes[0,0].plot(ax_data.values); axes[0,0].set_title('Original Series');plot_acf(ax_data.values, ax = axes[0,1]);

# # 1st Differencing
# axes[1,0].plot(diff_first); axes[1,0].set_title('1st order differencing');plot_acf((diff_first), ax = axes[1,1]);

# # 2nd Differencing
# axes[2,0].plot(diff_second); axes[2,0].set_title('2nd order differencing');plot_acf((diff_second), ax = axes[2,1]);


In [ ]:
plt.rcParams.update({'figure.figsize':(9,3), 'figure.dpi':120})

In [ ]:
fig, axes = plt.subplots(1, 2, sharex=True);axes[0].plot(diff_first); axes[0].set_title('1st Differencing');axes[1].set(ylim=(0,5))
plot_pacf(diff_second, ax=axes[1]);

**Plot ACF and PACF**

In [ ]:
from statsmodels.tsa.stattools import acf, pacf

In [ ]:
x = mean['y'][~np.isnan(mean['y'])]

lag_auto_corr = acf(x, nlags=7)
lag_par_auto_corr = pacf(x, nlags=7, method='ols')


#Plot ACF: 
plt.subplot(121) 
plt.plot(lag_auto_corr)
plt.axhline(y=0,linestyle='--',color='black')
plt.axhline(y=-1.96/np.sqrt(len(az_data)),linestyle='--',color='black')
plt.axhline(y=1.96/np.sqrt(len(az_data)),linestyle='--',color='black')
plt.title('Autocorrelation Function')

#Plot PACF:
plt.subplot(122)
plt.plot(lag_par_auto_corr)
plt.axhline(y=0,linestyle='--',color='black')
plt.axhline(y=-1.96/np.sqrt(len(az_data)),linestyle='--',color='black')
plt.axhline(y=1.96/np.sqrt(len(az_data)),linestyle='--',color='black')
plt.title('Partial Autocorrelation Function')
plt.tight_layout()

#**ARIMA**

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
model = ARIMA(az_data.values, order = (5,1,5))
model_fit = model.fit(disp=0,transparams=False)
print(model_fit.summary())

In [ ]:
model_fit.plot_predict(dynamic=False);


In [ ]:
model = ARIMA(ax_data.values, order = (1,1,1))
model_fit = model.fit(disp=0)
print(model_fit.summary())

In [ ]:
model_fit.plot_predict(dynamic=False)
plt.show()

**Rolling Mean and Rolling standard deviation**

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
def check_stationarity(timeseries):
    rolling_mean = timeseries.rolling(window=10,center=False).mean() 
    rolling_std = timeseries.rolling(window=10,center=False).std()

    #Plot rolling statistics:
    plt.ylim(0.99,1.01)
    original = plt.plot(timeseries.index.to_pydatetime(), timeseries.values, color='blue',label='Original')
    mean = plt.plot(rolling_mean.index.to_pydatetime(), rolling_mean.values, color='red', label='Rolling Mean')
    std = plt.plot(rolling_std.index.to_pydatetime(), rolling_std.values, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    
    #Perform Dickey-Fuller test:
    print ('Results of Dickey-Fuller Test:')
    dickey_fuller_test = adfuller(timeseries, autolag='AIC')
    dfresults = pd.Series(dickey_fuller_test[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dickey_fuller_test[4].items():
        dfresults['Critical Value (%s)'%key] = value
    print (dfresults)

In [ ]:
check_stationarity(ax_data)